# FinBERT Example Notebook

This notebooks shows how to train and use the FinBERT pre-trained language model for financial sentiment analysis.

## Modules 

In [1]:
from pathlib import Path
import shutil
import os
import logging
import sys
sys.path.append('..')

from textblob import TextBlob
from pprint import pprint
from sklearn.metrics import classification_report

from transformers import AutoModelForSequenceClassification

from finbert.finbert import *
import finbert.utils as tools

%load_ext autoreload
%autoreload 2

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', -1)

In [2]:
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.ERROR)

## Prepare the model

### Setting path variables:
1. `lm_path`: the path for the pre-trained language model (If vanilla Bert is used then no need to set this one).
2. `cl_path`: the path where the classification model is saved.
3. `cl_data_path`: the path of the directory that contains the data files of `train.csv`, `validation.csv`, `test.csv`.
---

In the initialization of `bertmodel`, we can either use the original pre-trained weights from Google by giving `bm = 'bert-base-uncased`, or our further pre-trained language model by `bm = lm_path`


---
All of the configurations with the model is controlled with the `config` variable. 

In [6]:
lm_path = project_dir/'models'/'language_model'/'bert-base-uncased'
cl_path = project_dir/'models'/'classifier_model'/'finbert-sentiment'
cl_data_path = project_dir/'data'/'sentiment_data'

###  Configuring training parameters

You can find the explanations of the training parameters in the class docsctrings. 

In [200]:
# Clean the cl_path
try:
    shutil.rmtree(cl_path) 
except:
    pass

#bertmodel = AutoModelForSequenceClassification.from_pretrained(lm_path,cache_dir=None, num_labels=3)
bertmodel = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)


config = Config(   data_dir=cl_data_path,
                   bert_model=bertmodel,
                   num_train_epochs=10,
                   model_dir=cl_path,
                   max_seq_length = 48,
                   train_batch_size = 32,
                   learning_rate = 2e-5,
                   output_mode='classification',
                   warm_up_proportion=0.2,
                   local_rank=-1,
                   discriminate=True,
                   gradual_unfreeze=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

`finbert` is our main class that encapsulates all the functionality. The list of class labels should be given in the prepare_model method call with label_list parameter.

In [201]:
finbert = FinBert(config)
finbert.base_model = 'bert-base-uncased'
finbert.config.discriminate=True
finbert.config.gradual_unfreeze=True

In [202]:
finbert.prepare_model(label_list=['positive','negative','neutral'])

11/04/2021 14:04:55 - INFO - finbert.finbert -   device: cuda n_gpu: 2, distributed training: False, 16-bits training: False


## Fine-tune the model

In [203]:
# Get the training examples
train_data = finbert.get_data('train')

In [204]:
model = finbert.create_the_model()

In [205]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_aff

### [Optional] Fine-tune only a subset of the model
The variable `freeze` determines the last layer (out of 12) to be freezed. You can skip this part if you want to fine-tune the whole model.

<span style="color:red">Important: </span>
Execute this step if you want a shorter training time in the expense of accuracy.

In [28]:
# This is for fine-tuning a subset of the model.

freeze = 6

for param in model.bert.embeddings.parameters():
    param.requires_grad = False
    
for i in range(freeze):
    for param in model.bert.encoder.layer[i].parameters():
        param.requires_grad = False

### Training

In [206]:
trained_model = finbert.train(train_examples = train_data, model = model)

11/04/2021 14:05:17 - INFO - finbert.utils -   *** Example ***
11/04/2021 14:05:17 - INFO - finbert.utils -   guid: train-1
11/04/2021 14:05:17 - INFO - finbert.utils -   tokens: [CLS] the technology will become available to businesses from the fourth quarter , nokia said [SEP]
11/04/2021 14:05:17 - INFO - finbert.utils -   input_ids: 101 1996 2974 2097 2468 2800 2000 5661 2013 1996 2959 4284 1010 22098 2056 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/04/2021 14:05:17 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/04/2021 14:05:17 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/04/2021 14:05:17 - INFO - finbert.utils -   label: neutral (id = 2)
11/04/2021 14:05:18 - INFO - finbert.finbert -   ***** Loading data *****
11/04/2021 14:05:18 - INFO - finbert.finbert -     Num ex

11/04/2021 14:05:47 - INFO - finbert.utils -   *** Example ***
11/04/2021 14:05:47 - INFO - finbert.utils -   guid: validation-1
11/04/2021 14:05:47 - INFO - finbert.utils -   tokens: [CLS] no blind - spots coming from 1 vantage point allows investigators to see the whole story in high resolution with no unbroken video [SEP]
11/04/2021 14:05:47 - INFO - finbert.utils -   input_ids: 101 2053 6397 1011 7516 2746 2013 1015 27274 2391 4473 14766 2000 2156 1996 2878 2466 1999 2152 5813 2007 2053 29505 2678 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/04/2021 14:05:47 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/04/2021 14:05:47 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/04/2021 14:05:47 - INFO - finbert.utils -   label: neutral (id = 2)
11/04/2021 14:05:48 - INFO - finbert.finbert -   *****


Validation losses: [0.5441211743934734]
No best model found


Epoch:  10%|████████                                                                         | 1/10 [00:32<04:50, 32.32s/it]

11/04/2021 14:06:36 - INFO - finbert.utils -   *** Example ***
11/04/2021 14:06:36 - INFO - finbert.utils -   guid: validation-1
11/04/2021 14:06:36 - INFO - finbert.utils -   tokens: [CLS] no blind - spots coming from 1 vantage point allows investigators to see the whole story in high resolution with no unbroken video [SEP]
11/04/2021 14:06:36 - INFO - finbert.utils -   input_ids: 101 2053 6397 1011 7516 2746 2013 1015 27274 2391 4473 14766 2000 2156 1996 2878 2466 1999 2152 5813 2007 2053 29505 2678 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/04/2021 14:06:36 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/04/2021 14:06:36 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/04/2021 14:06:36 - INFO - finbert.utils -   label: neutral (id = 2)
11/04/2021 14:06:36 - INFO - finbert.finbert -   *****

...

Epoch:  90%|████████████████████████████████████████████████████████████████████████▉        | 9/10 [11:07<01:22, 82.64s/it]


Validation losses: [0.5441211743934734, 0.22304185137555405, 0.15144505092521776, 0.13746784529271158, 0.13465465228325008, 0.14080110450300412, 0.14677860233869805, 0.16414479311631136, 0.15525638365589484]


11/04/2021 14:17:50 - INFO - finbert.utils -   *** Example ***
11/04/2021 14:17:50 - INFO - finbert.utils -   guid: validation-1
11/04/2021 14:17:50 - INFO - finbert.utils -   tokens: [CLS] no blind - spots coming from 1 vantage point allows investigators to see the whole story in high resolution with no unbroken video [SEP]
11/04/2021 14:17:50 - INFO - finbert.utils -   input_ids: 101 2053 6397 1011 7516 2746 2013 1015 27274 2391 4473 14766 2000 2156 1996 2878 2466 1999 2152 5813 2007 2053 29505 2678 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/04/2021 14:17:50 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/04/2021 14:17:50 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/04/2021 14:17:50 - INFO - finbert.utils -   label: neutral (id = 2)
11/04/2021 14:17:50 - INFO - finbert.finbert -   *****

Epoch: 100%|████████████████████████████████████████████████████████████████████████████████| 10/10 [12:34<00:00, 75.46s/it]


Validation losses: [0.5441211743934734, 0.22304185137555405, 0.15144505092521776, 0.13746784529271158, 0.13465465228325008, 0.14080110450300412, 0.14677860233869805, 0.16414479311631136, 0.15525638365589484, 0.1488779369187919]


In [116]:
t = torch.rand(32, 48)
t.view(-1, 3)


tensor([[0.2671, 0.7901, 0.5045],
        [0.2648, 0.8128, 0.7379],
        [0.5538, 0.1281, 0.7425],
        ...,
        [0.2638, 0.4249, 0.0092],
        [0.7504, 0.1800, 0.7006],
        [0.5807, 0.6974, 0.8930]])

## Test the model

`bert.evaluate` outputs the DataFrame, where true labels and logit values for each example is given

In [183]:
test_data = finbert.get_data('test')

In [184]:
results = finbert.evaluate(examples=test_data, model=trained_model)

11/04/2021 10:55:52 - INFO - finbert.utils -   *** Example ***
11/04/2021 10:55:52 - INFO - finbert.utils -   guid: test-1
11/04/2021 10:55:52 - INFO - finbert.utils -   tokens: [CLS] demand was brisk as expected and order levels have remained high [SEP]
11/04/2021 10:55:52 - INFO - finbert.utils -   input_ids: 101 5157 2001 28022 2004 3517 1998 2344 3798 2031 2815 2152 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/04/2021 10:55:52 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/04/2021 10:55:52 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/04/2021 10:55:52 - INFO - finbert.utils -   label: positive (id = 0)
11/04/2021 10:55:52 - INFO - finbert.finbert -   ***** Loading data *****
11/04/2021 10:55:52 - INFO - finbert.finbert -     Num examples = 2956
11/04/2021 10:55:5

### Prepare the classification report

In [207]:
def report(df, cols=['label','prediction','logits']):
    #print('Validation loss:{0:.2f}'.format(metrics['best_validation_loss']))
    cs = CrossEntropyLoss(weight=finbert.class_weights)
    loss = cs(torch.tensor(list(df[cols[2]])),torch.tensor(list(df[cols[0]])))
    print("Loss:{0:.2f}".format(loss))
    print("Accuracy:{0:.2f}".format((df[cols[0]] == df[cols[1]]).sum() / df.shape[0]) )
    print("\nClassification Report:")
    print(classification_report(df[cols[0]], df[cols[1]]))

In [208]:
results['prediction'] = results.predictions.apply(lambda x: np.argmax(x,axis=0))

In [210]:
report(results,cols=['labels','prediction','predictions'])

Loss:0.14
Accuracy:0.96

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.95      0.95       814
           1       0.92      0.96      0.94       345
           2       0.97      0.97      0.97      1797

    accuracy                           0.96      2956
   macro avg       0.95      0.96      0.95      2956
weighted avg       0.96      0.96      0.96      2956



### Get predictions

With the `predict` function, given a piece of text, we split it into a list of sentences and then predict sentiment for each sentence. The output is written into a dataframe. Predictions are represented in three different columns: 

1) `logit`: probabilities for each class

2) `prediction`: predicted label

3) `sentiment_score`: sentiment score calculated as: probability of positive - probability of negative

Below we analyze a paragraph taken out of [this](https://www.economist.com/finance-and-economics/2019/01/03/a-profit-warning-from-apple-jolts-markets) article from The Economist. For comparison purposes, we also put the sentiments predicted with TextBlob.
> Later that day Apple said it was revising down its earnings expectations in the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China. The news rapidly infected financial markets. Apple’s share price fell by around 7% in after-hours trading and the decline was extended to more than 10% when the market opened. The dollar fell by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering some ground. Asian stockmarkets closed down on January 3rd and European ones opened lower. Yields on government bonds fell as investors fled to the traditional haven in a market storm.

In [188]:
text = "Later that day Apple said it was revising down its earnings expectations in \
the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China. \
The news rapidly infected financial markets. Apple’s share price fell by around 7% in after-hours \
trading and the decline was extended to more than 10% when the market opened. The dollar fell \
by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering \
some ground. Asian stockmarkets closed down on January 3rd and European ones opened lower. \
Yields on government bonds fell as investors fled to the traditional haven in a market storm."

In [189]:
cl_path = project_dir/'models'/'classifier_model'/'finbert-sentiment'
model = AutoModelForSequenceClassification.from_pretrained(cl_path, cache_dir=None, num_labels=3)

In [139]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Ari
[nltk_data]     Shater\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [190]:
result = predict(text,model)

11/04/2021 10:56:25 - INFO - finbert.utils -   *** Example ***
11/04/2021 10:56:25 - INFO - finbert.utils -   guid: 0
11/04/2021 10:56:25 - INFO - finbert.utils -   tokens: [CLS] later that day apple said it was rev ##ising down its earnings expectations in the fourth quarter of 2018 , largely because of lower sales and signs of economic weakness in china . [SEP]
11/04/2021 10:56:25 - INFO - finbert.utils -   input_ids: 101 2101 2008 2154 6207 2056 2009 2001 7065 9355 2091 2049 16565 10908 1999 1996 2959 4284 1997 2760 1010 4321 2138 1997 2896 4341 1998 5751 1997 3171 11251 1999 2859 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/04/2021 10:56:25 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/04/2021 10:56:25 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [192]:
blob = TextBlob(text)
result['textblob_prediction'] = [sentence.sentiment.polarity for sentence in blob.sentences]
result

,sentence,logit,prediction,sentiment_score,textblob_prediction
0,"Later that day Apple said it was revising down its earnings expectations in the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China.","[0.0033345295, 0.98776615, 0.008899213]",negative,-0.984432,0.051746
1,The news rapidly infected financial markets.,"[0.008665337, 0.96750736, 0.023827318]",negative,-0.958842,0.000000
2,Apple’s share price fell by around 7% in after-hours trading and the decline was extended to more than 10% when the market opened.,"[0.0026462073, 0.99120516, 0.0061486457]",negative,-0.988559,0.500000
3,"The dollar fell by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering some ground.","[0.007249232, 0.9876209, 0.0051299236]",negative,-0.980372,0.000000
4,Asian stockmarkets closed down on January 3rd and European ones opened lower.,"[0.0038619624, 0.9161373, 0.08000076]",negative,-0.912275,-0.051111
5,Yields on government bonds fell as investors fled to the traditional haven in a market storm.,"[0.0039962516, 0.9946044, 0.0013992615]",negative,-0.990608,0.000000


In [193]:
print(f'Average sentiment is %.2f.' % (result.sentiment_score.mean()))

Average sentiment is -0.97.


Here is another example

In [194]:
text2 = "Shares in the spin-off of South African e-commerce group Naspers surged more than 25% \
in the first minutes of their market debut in Amsterdam on Wednesday. Bob van Dijk, CEO of \
Naspers and Prosus Group poses at Amsterdam's stock exchange, as Prosus begins trading on the \
Euronext stock exchange in Amsterdam, Netherlands, September 11, 2019. REUTERS/Piroschka van de Wouw \
Prosus comprises Naspers’ global empire of consumer internet assets, with the jewel in the crown a \
31% stake in Chinese tech titan Tencent. There is 'way more demand than is even available, so that’s \
good,' said the CEO of Euronext Amsterdam, Maurice van Tilburg. 'It’s going to be an interesting \
hour of trade after opening this morning.' Euronext had given an indicative price of 58.70 euros \
per share for Prosus, implying a market value of 95.3 billion euros ($105 billion). The shares \
jumped to 76 euros on opening and were trading at 75 euros at 0719 GMT."

In [195]:
result2 = predict(text2,model)
blob = TextBlob(text2)
result2['textblob_prediction'] = [sentence.sentiment.polarity for sentence in blob.sentences]

11/04/2021 10:56:34 - INFO - finbert.utils -   *** Example ***
11/04/2021 10:56:34 - INFO - finbert.utils -   guid: 0
11/04/2021 10:56:34 - INFO - finbert.utils -   tokens: [CLS] shares in the spin - off of south african e - commerce group nas ##pers surged more than 25 % in the first minutes of their market debut in amsterdam on wednesday . [SEP]
11/04/2021 10:56:34 - INFO - finbert.utils -   input_ids: 101 6661 1999 1996 6714 1011 2125 1997 2148 3060 1041 1011 6236 2177 17235 7347 18852 2062 2084 2423 1003 1999 1996 2034 2781 1997 2037 3006 2834 1999 7598 2006 9317 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/04/2021 10:56:34 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/04/2021 10:56:34 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [196]:
result2

,sentence,logit,prediction,sentiment_score,textblob_prediction
0,Shares in the spin-off of South African e-commerce group Naspers surged more than 25% in the first minutes of their market debut in Amsterdam on Wednesday.,"[0.9920455, 0.004372825, 0.0035817088]",positive,0.987673,0.250000
1,"Bob van Dijk, CEO of Naspers and Prosus Group poses at Amsterdam's stock exchange, as Prosus begins trading on the Euronext stock exchange in Amsterdam, Netherlands, September 11, 2019.","[0.0037315853, 0.0064404076, 0.989828]",neutral,-0.002709,0.000000
2,"REUTERS/Piroschka van de Wouw Prosus comprises Naspers’ global empire of consumer internet assets, with the jewel in the crown a 31% stake in Chinese tech titan Tencent.","[0.0032723187, 0.0016629169, 0.9950648]",neutral,0.001609,0.000000
3,"There is 'way more demand than is even available, so that’s good,' said the CEO of Euronext Amsterdam, Maurice van Tilburg.","[0.98820764, 0.0028737478, 0.008918695]",positive,0.985334,0.533333
4,'It’s going to be an interesting hour of trade after opening this morning.',"[0.014273786, 0.004459984, 0.98126626]",neutral,0.009814,0.500000
5,"Euronext had given an indicative price of 58.70 euros per share for Prosus, implying a market value of 95.3 billion euros ($105 billion).","[0.0059209173, 0.00090549554, 0.99317354]",neutral,0.005015,0.000000
6,The shares jumped to 76 euros on opening and were trading at 75 euros at 0719 GMT.,"[0.21073681, 0.002572727, 0.7866905]",neutral,0.208164,0.000000


In [198]:
print(f'Average sentiment is %.2f.' % (result2.sentiment_score.mean()))

Average sentiment is 0.31.
